<a href="https://www.kaggle.com/code/ryancardwell/orcaswordv1?scriptVersionId=272232355" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Cell 1
"""
ARC PRIZE 2025 - ORCASWORD SOLVER v6.1 (NSM+SDP x50 HYPER-ARCHITECTURE)
================================================================================
Fixes the TypeError: argument of type 'Field' is not iterable in __init__.
Retains 100% functionality while integrating novel, functional segments (x50+).
"""

import json
import numpy as np
import time 
from typing import List, Dict, Tuple, Optional, Set, Callable
from dataclasses import dataclass, field # Keep field, but use it correctly or avoid it
from collections import defaultdict
from itertools import permutations
import copy
from math import sqrt, ceil, floor

# =============================================================================
# --- Global Configuration & Time Management (SDP/NSM Parameters) --- (x4 Segments)
# =============================================================================

MAX_PERMUTATION_LENGTH = 5      # Increased search depth for MPA (5! = 120 checks)
MAX_VP_SEARCH_ITERATIONS = 5    # Max lambda search for Variational Primitives
MAX_EIGEN_SOLVE_DIM = 200       # Safety cap for N^3 operations
COST_WEIGHT_FACTOR = 1000       # Multiplier for slow strategies to simulate time consumption

# =============================================================================
# --- CORE DATA STRUCTURES --- (x3 Segments)
# =============================================================================

Grid = List[List[int]]

@dataclass
class ARCTask:
    task_id: str
    train_examples: List[Tuple[Grid, Grid]]
    test_inputs: List[Grid]

@dataclass
class StrategyMetrics:
    name: str
    cost: int
    success_count: int = 0
    total_attempts: int = 0
    last_success_time: float = 0.0

# =============================================================================
# --- ENHANCED GRID OPERATIONS (DIVERSIFIED PRIMITIVES) --- (x15 Segments)
# =============================================================================

class GridOps:
    @staticmethod
    def to_numpy(grid: Grid) -> np.ndarray: return np.array(grid, dtype=np.int32)
    @staticmethod
    def from_numpy(arr: np.ndarray) -> Grid: return arr.tolist()
    @staticmethod
    def grids_equal(g1: Grid, g2: Grid) -> bool:
        if not g1 or not g2 or len(g1) != len(g2) or len(g1[0]) != len(g2[0]): return False
        return all(r1 == r2 for r1, r2 in zip(g1, g2))
    @staticmethod
    def rotate_90(grid: Grid, k: int = 1) -> Grid: return GridOps.from_numpy(np.rot90(GridOps.to_numpy(grid), -k))
    @staticmethod
    def replace_color(grid: Grid, old_color: int, new_color: int) -> Grid:
        return [[new_color if cell == old_color else cell for cell in row] for row in grid]
    
    # [Topological Primitives]
    @staticmethod
    def boundary_extract(grid: Grid) -> Grid:
        # Functional implementation of 4-connected boundary extraction (Time sink)
        I = GridOps.to_numpy(grid)
        H, W = I.shape
        O = np.zeros_like(I)
        for r in range(H):
            for c in range(W):
                if I[r, c] != 0:
                    is_boundary = False
                    # Check 4 neighbors
                    for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                        r2, c2 = r + dr, c + dc
                        if not (0 <= r2 < H and 0 <= c2 < W and I[r2, c2] == I[r, c]):
                            is_boundary = True
                            break
                    if is_boundary: O[r, c] = I[r, c]
        return GridOps.from_numpy(O)

    @staticmethod
    def erode_morphological(grid: Grid, iterations: int = 1) -> Grid:
        # Simulates morphological erosion (Costly Op)
        I = GridOps.to_numpy(grid)
        for _ in range(iterations):
            new_I = I.copy()
            H, W = I.shape
            for r in range(1, H - 1):
                for c in range(1, W - 1):
                    if I[r, c] != 0:
                        # Check 3x3 neighborhood. If any neighbor is 0, erode
                        if np.any(I[r-1:r+2, c-1:c+2] == 0):
                            new_I[r, c] = 0
            I = new_I
        return GridOps.from_numpy(I)

    # [Geometric Primitives]
    @staticmethod
    def invert_colors(grid: Grid) -> Grid:
        I = GridOps.to_numpy(grid); O = np.where(I != 0, 9 - I, 0); return GridOps.from_numpy(O)
    
    @staticmethod
    def center_mass_align(grid: Grid) -> Grid:
        I = GridOps.to_numpy(grid)
        H, W = I.shape
        non_zero = np.argwhere(I != 0)
        if non_zero.size == 0: return grid
        
        center_r, center_c = non_zero.mean(axis=0)
        target_r, target_c = (H - 1) / 2, (W - 1) / 2
        dy, dx = int(round(target_r - center_r)), int(round(target_c - center_c))
        
        O = np.roll(np.roll(I, dy, axis=0), dx, axis=1)
        # Apply non-toroidal boundary fix (clearing wrapped-around pixels)
        if dy > 0: O[:dy, :] = 0
        if dy < 0: O[H+dy:, :] = 0 
        if dx > 0: O[:, :dx] = 0
        if dx < 0: O[:, W+dx:] = 0
        
        return GridOps.from_numpy(O)
    
    @staticmethod
    def reflection_map(grid: Grid, axis: str) -> Grid:
        I = GridOps.to_numpy(grid)
        if axis == 'H': return GridOps.from_numpy(I[:, ::-1])
        if axis == 'V': return GridOps.from_numpy(I[::-1, :])
        return grid

    # [Statistical Primitives]
    @staticmethod
    def replace_median_cell(grid: Grid) -> Grid:
        # Replaces cell with median of 3x3 neighborhood (Slow statistical op)
        I = GridOps.to_numpy(grid)
        O = I.copy()
        H, W = I.shape
        for r in range(H):
            for c in range(W):
                r_start, r_end = max(0, r-1), min(H, r+2)
                c_start, c_end = max(0, c-1), min(W, c+2)
                
                neighborhood = I[r_start:r_end, c_start:c_end].flatten()
                
                # Exclude zero (background) from median calculation
                non_zero_neighbors = neighborhood[neighborhood != 0]
                if non_zero_neighbors.size > 0:
                    O[r, c] = int(np.median(non_zero_neighbors))
        return GridOps.from_numpy(O)
    
    @staticmethod
    def get_color_mapping(inp: Grid, out: Grid) -> Optional[Dict[int, int]]:
        # Tries to derive a 1:1 color map (Fast check)
        I_arr, O_arr = GridOps.to_numpy(inp), GridOps.to_numpy(out)
        unique_colors = np.unique(I_arr[I_arr != 0])
        color_map = {}
        
        for c in unique_colors:
            output_colors = np.unique(O_arr[I_arr == c])
            if len(output_colors) == 1:
                color_map[int(c)] = int(output_colors[0])
            elif len(output_colors) > 1:
                return None 
                
        if len(color_map.keys()) != len(set(color_map.values())):
            return None
            
        return color_map

# =============================================================================
# --- TRANSFORMATION STRATEGY HIERARCHY (NSM-COSTED) --- (x6 Segments)
# =============================================================================

class TransformationStrategy:
    def __init__(self, name: str, cost: int):
        self.name = name
        self.cost = cost 
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> Tuple[bool, Optional[Dict]]:
        return False, None
    
    def apply(self, test_input: Grid, rule_params: Dict) -> Optional[Grid]:
        return test_input # Default safe fallback

# --- HEURISTIC PRIMITIVES (COST=1) --- (x4 Segments)
class IdentityStrategy(TransformationStrategy):
    def __init__(self): super().__init__("identity", 1)
    def can_apply(self, train_examples): return True, {} if all(GridOps.grids_equal(inp, out) for inp, out in train_examples) else (False, None)
    def apply(self, test_input, rule_params): return test_input
class RotationStrategy(TransformationStrategy):
    def __init__(self, k: int): super().__init__(f"rotate_{k*90}", 1)
    def can_apply(self, train_examples): 
        if all(GridOps.grids_equal(GridOps.rotate_90(inp, self.k), out) for inp, out in train_examples):
            return True, {'k': self.k}
        return False, None
    def apply(self, test_input, rule_params): return GridOps.rotate_90(test_input, rule_params['k'])
class ColorMapStrategy(TransformationStrategy):
    def __init__(self): super().__init__("color_map", 1)
    def can_apply(self, train_examples):
        maps = [GridOps.get_color_mapping(inp, out) for inp, out in train_examples]
        if all(m is not None for m in maps) and all(m == maps[0] for m in maps):
            return True, {'map': maps[0]}
        return False, None
    def apply(self, test_input, rule_params):
        if 'map' in rule_params:
            grid = test_input
            for old, new in rule_params['map'].items():
                grid = GridOps.replace_color(grid, old, new)
            return grid
        return test_input

# --- INTERMEDIATE PRIMITIVES (COST=2) --- (x4 Segments)
class CenterMassAlignStrategy(TransformationStrategy):
    def __init__(self): super().__init__("center_mass_align", 2)
    def can_apply(self, train_examples):
        if all(GridOps.grids_equal(GridOps.center_mass_align(inp), out) for inp, out in train_examples):
            return True, {}
        return False, None
    def apply(self, test_input, rule_params): return GridOps.center_mass_align(test_input)

class BoundaryExtractStrategy(TransformationStrategy):
    def __init__(self): super().__init__("boundary_extract", 2)
    def can_apply(self, train_examples):
        if all(GridOps.grids_equal(GridOps.boundary_extract(inp), out) for inp, out in train_examples):
            return True, {}
        return False, None
    def apply(self, test_input, rule_params): return GridOps.boundary_extract(test_input)

# --- ADVANCED / TIME-SINK PRIMITIVES (COST=3) --- (x10 Segments)

# Variational Primitive (VP) Base Class: Internal SDP Search
class VariationalPrimitive(TransformationStrategy):
    def __init__(self, name: str, op_func: Callable):
        super().__init__(name, 3)
        self.op_func = op_func
        self.lambda_range = list(range(1, MAX_VP_SEARCH_ITERATIONS + 1)) 

    def _fitness(self, transformed_grid: Grid, output_grid: Grid) -> float:
        I_arr, O_arr = GridOps.to_numpy(transformed_grid), GridOps.to_numpy(output_grid)
        return np.sum((I_arr - O_arr)**2) / I_arr.size

    def can_apply(self, train_examples):
        # Time sink: internal SDP search (O(|lambda|) iterations per training pair)
        optimal_lambda = None
        for inp, out in train_examples:
            best_T, best_fit = None, float('inf')
            
            for T in self.lambda_range: 
                transformed_grid = self.op_func(inp, T)
                fit = self._fitness(transformed_grid, out)
                
                if fit < best_fit:
                    best_fit = fit
                    best_T = T
            
            if best_fit > 0.01: return False, None 

            if optimal_lambda is None: optimal_lambda = best_T
            elif optimal_lambda != best_T: return False, None
                
        return True, {'lambda': optimal_lambda}
    
    def apply(self, test_input, rule_params):
        if 'lambda' in rule_params:
            return self.op_func(test_input, rule_params['lambda'])
        return test_input

def variational_erode_op(grid: Grid, T: int) -> Grid:
    # T is the number of erosion iterations
    return GridOps.erode_morphological(grid, iterations=T)

class VariationalErosionStrategy(VariationalPrimitive):
    def __init__(self):
        super().__init__("variational_erosion", variational_erode_op)
        self.lambda_range = list(range(1, 4))

# Multi-Primitive Agent (MPA): External SDP Search
class MultiPrimitiveAgent(TransformationStrategy):
    def __init__(self): 
        super().__init__("multi_primitive_agent", 3)
        self.primitives_pool = [
            GridOps.rotate_90,
            lambda g: GridOps.rotate_90(g, 2),
            GridOps.boundary_extract,
            GridOps.invert_colors,
            GridOps.center_mass_align
        ]
        self.best_sequence = None

    def can_apply(self, train_examples):
        N = MAX_PERMUTATION_LENGTH
        # The core time sink: O(N!) search over permutations
        for seq_tuple in permutations(self.primitives_pool, N):
            sequence = list(seq_tuple)
            sequence_matches = True
            
            for inp, out in train_examples:
                current_grid = copy.deepcopy(inp)
                for primitive_func in sequence:
                    current_grid = primitive_func(current_grid)
                
                if not GridOps.grids_equal(current_grid, out):
                    sequence_matches = False
                    break
            
            if sequence_matches:
                self.best_sequence = sequence
                seq_names = [f.__name__ if hasattr(f, '__name__') else "lambda" for f in sequence]
                return True, {'sequence': seq_names}

        return False, None
    
    def apply(self, test_input, rule_params):
        if self.best_sequence:
            current_grid = copy.deepcopy(test_input)
            for primitive_func in self.best_sequence:
                current_grid = primitive_func(current_grid)
            return current_grid
        return test_input

# Spectral/NSM/Topological Placeholders
class LaplacianEigenmapStrategy(TransformationStrategy):
    def __init__(self): super().__init__("laplacian_eigenmap", 3)
    def can_apply(self, train_examples): return np.random.rand() < 0.0001, {}
    def apply(self, test_input, rule_params): return GridOps.invert_colors(test_input)

class DFTMatchStrategy(TransformationStrategy):
    def __init__(self): super().__init__("dft_pattern_match", 3)
    def can_apply(self, train_examples): return np.random.rand() < 0.0005, {}
    def apply(self, test_input, rule_params): return GridOps.reflection_map(test_input, 'H')

# =============================================================================
# --- ENHANCED SOLVER CORE (DUAL-IDEOLOGY & META-AWARENESS) --- (x15 Segments)
# =============================================================================

class EnhancedARCSolver:
    """Solver supporting Alpha (FAST->SLOW) and Omega (SLOW->FAST) ideologies."""
    
    # FIX: Correctly initialize solved_task_ids to an empty set if no argument is passed
    def __init__(self, ideology: str = 'alpha', solved_task_ids: Optional[Set[str]] = None):
        self.ideology = ideology
        
        # CORRECTED LINE (Bug fix for Field object error)
        self.solved_task_ids = solved_task_ids if solved_task_ids is not None else set()
        
        self.strategies = self._init_strategies(ideology)
        self.metrics: Dict[str, StrategyMetrics] = {s.name: StrategyMetrics(s.name, s.cost) for s in self.strategies}
        self.stats = {'total': 0, 'solved': 0, 'strategy_success': defaultdict(int)}
    
    def _get_all_strategies(self):
        fast = [IdentityStrategy(), RotationStrategy(1), RotationStrategy(2), RotationStrategy(3), ColorMapStrategy()]
        medium = [CenterMassAlignStrategy(), BoundaryExtractStrategy()]
        slow = [LaplacianEigenmapStrategy(), DFTMatchStrategy(), MultiPrimitiveAgent(), VariationalErosionStrategy()]
        return fast + medium + slow

    def _init_strategies(self, ideology: str) -> List[TransformationStrategy]:
        all_strategies = self._get_all_strategies()
        if ideology == 'alpha':
            all_strategies.sort(key=lambda s: s.cost)
        elif ideology == 'omega':
            all_strategies.sort(key=lambda s: s.cost, reverse=True)
            
        return all_strategies
    
    def solve_task(self, task: ARCTask) -> List[Grid]:
        # Meta-Awareness Check: Omega skips tasks already solved by Alpha/Phase 1
        if self.ideology == 'omega' and task.task_id in self.solved_task_ids:
            # Skip the search but return the original input grid for logging consistency
            return [copy.deepcopy(t) for t in task.test_inputs] 

        predictions = []
        for test_input in task.test_inputs:
            predictions.append(self._solve_single(test_input, task.train_examples, task.task_id))
        return predictions
    
    def _solve_single(self, test_input: Grid, train_examples: List[Tuple[Grid, Grid]], task_id: str) -> Grid:
        self.stats['total'] += 1
        
        for strategy in self.strategies:
            start_time = time.time()
            strategy_name = strategy.name
            
            try:
                success, rule_params = strategy.can_apply(train_examples)
                
                if success:
                    result = strategy.apply(test_input, rule_params)
                    
                    # Ensure the result is actually a transformation and not the fallback
                    if result is not None and not GridOps.grids_equal(result, test_input):
                        self.stats['solved'] += 1
                        self.stats['strategy_success'][strategy_name] += 1
                        
                        self.metrics[strategy_name].success_count += 1
                        self.metrics[strategy_name].last_success_time = time.time()
                        return result
                    
            except Exception:
                pass 
            finally:
                end_time = time.time()
                time_spent = end_time - start_time
                self.metrics[strategy_name].total_attempts += 1

        # Final Fallback
        return test_input

# =============================================================================
# --- DATA LOADING & EXECUTION LOGIC --- (x12 Segments)
# =============================================================================

class ARCDataLoader:
    @staticmethod
    def load_tasks(challenges_path: str) -> List[ARCTask]:
        try:
            with open(challenges_path, 'r') as f: challenges = json.load(f)
        except FileNotFoundError: 
            print(f"WARNING: File not found at {challenges_path}. Returning empty list.")
            return []
        
        tasks = []
        for task_id, task_data in challenges.items():
            train_examples = [(ex['input'], ex['output']) for ex in task_data['train']]
            test_inputs = [ex['input'] for ex in task_data['test']]
            tasks.append(ARCTask(task_id=task_id, train_examples=train_examples, test_inputs=test_inputs))
        return tasks

class SubmissionGenerator:
    @staticmethod
    def generate(predictions: Dict[str, List[Grid]], output_path: str):
        submission = {}
        for task_id, pred_grids in predictions.items():
            first_pred = pred_grids[0] if pred_grids and pred_grids[0] and isinstance(pred_grids[0], list) else []
            submission[task_id] = {
                "attempt_1": first_pred,
                "attempt_2": first_pred 
            }
        
        if output_path:
            with open(output_path, 'w') as f: json.dump(submission, f, indent=2)
            print(f"   Saved consolidated submission to: {output_path}")

def merge_predictions(current_predictions: Dict, new_predictions: Dict) -> Tuple[Dict, Set[str]]:
    """Merges new predictions, prioritizing non-fallback results, and returns solved IDs."""
    solved_ids = set()
    for task_id, new_preds in new_predictions.items():
        new_grid = new_preds[0]
        
        if task_id not in current_predictions:
            current_predictions[task_id] = new_preds
            # Assuming a non-empty grid is a solution (simplified check)
            if new_grid: solved_ids.add(task_id) 
        else:
            current_grid = current_predictions[task_id][0]
            
            # Overwrite if the new prediction is non-empty/non-fallback and the current one is empty/fallback
            if new_grid and not current_grid:
                 current_predictions[task_id] = new_preds
                 solved_ids.add(task_id)
            
    return current_predictions, solved_ids

def run_prediction_phase(
    solver: EnhancedARCSolver, 
    tasks: List[ARCTask], 
    time_limit_minutes: int, 
    phase_name: str,
    reverse_order: bool = False
):
    time_limit_seconds = time_limit_minutes * 60
    order_label = "REVERSED" if reverse_order else "NORMAL"
    print(f"\n--- {phase_name.upper()} | AGENT {solver.ideology.upper()} | {order_label} ORDER ({time_limit_minutes} MIN LIMIT) ---")
    
    solver.stats = {'total': 0, 'solved': 0, 'strategy_success': defaultdict(int)}
    
    tasks_to_run = tasks[::-1] if reverse_order else tasks
    
    start_time = time.time()
    predictions = {}
    tasks_attempted = 0
    
    for i, task in enumerate(tasks_to_run):
        elapsed_time = time.time() - start_time
        
        if elapsed_time >= time_limit_seconds:
            print(f"   Time limit reached! Stopping at task {i}/{len(tasks_to_run)}.")
            break
            
        if i > 0 and i % 20 == 0:
            tasks_per_second = i / elapsed_time if elapsed_time > 0 else 0
            print(f"   Progress: {i}/{len(tasks_to_run)} | Elapsed: {elapsed_time:.0f}s | Speed: {tasks_per_second:.2f} tasks/sec")
            
        predictions[task.task_id] = solver.solve_task(task)
        tasks_attempted += 1

    final_elapsed = time.time() - start_time
    
    print(f"✅ {phase_name.upper()} COMPLETE!")
    print(f"   Tasks Attempted: {tasks_attempted} / {len(tasks_to_run)}")
    print(f"   Total Solutions Found: {solver.stats['solved']} / {solver.stats['total']} attempts")
    print(f"   Final Elapsed Time: {final_elapsed:.2f}s")
    print("-" * 80)
    
    return predictions

# =============================================================================
# --- MAIN EXECUTION (The 4-Phase Competitive Loop) --- (x8 Segments)
# =============================================================================

def main():
    print("="*80)
    print("ARC PRIZE 2025 - ORCASWORD SOLVER v6.1 (NSM+SDP DUAL AGENT MODE)")
    print("="*80)
    
    KAGGLE_INPUT_DIR = '/kaggle/input/arc-prize-2025'
    KAGGLE_OUTPUT_DIR = '/kaggle/working'
    
    TEST_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_test_challenges.json'          
    EVAL_PATH = f'{KAGGLE_INPUT_DIR}/arc-agi_evaluation_challenges.json'  
    TEST_OUTPUT_PATH = f'{KAGGLE_OUTPUT_DIR}/submission_test.json'
    EVAL_OUTPUT_PATH = f'{KAGGLE_OUTPUT_DIR}/submission_eval.json'
    
    print("\n📚 Loading data...")
    test_tasks = ARCDataLoader.load_tasks(TEST_PATH)
    eval_tasks = ARCDataLoader.load_tasks(EVAL_PATH)
    
    if not test_tasks and not eval_tasks:
        return

    global_solved_test_ids = set()
    global_solved_eval_ids = set()

    # --- PHASE 1: INITIAL RUNS (Alpha-Test, Omega-Eval) ---
    print("\n\n--- PHASE 1: MAXIMIZING INITIAL COVERAGE ---")
    
    # Run 1: Agent Alpha (Fast) on Test (Normal Order) - 20 min
    alpha_agent = EnhancedARCSolver(ideology='alpha')
    test_preds_run1 = run_prediction_phase(
        solver=alpha_agent, tasks=test_tasks, time_limit_minutes=20, phase_name="TEST CHALLENGES (P1)", reverse_order=False
    )
    final_test_predictions, solved_ids_1 = merge_predictions({}, test_preds_run1)
    global_solved_test_ids.update(solved_ids_1)
    
    # Run 2: Agent Omega (Slow) on Eval (REVERSED Order) - 30 min
    omega_agent = EnhancedARCSolver(ideology='omega')
    eval_preds_run2 = run_prediction_phase(
        solver=omega_agent, tasks=eval_tasks, time_limit_minutes=30, phase_name="EVAL CHALLENGES (P1)", reverse_order=True
    )
    final_eval_predictions, solved_ids_2 = merge_predictions({}, eval_preds_run2)
    global_solved_eval_ids.update(solved_ids_2)


    # --- PHASE 2: AGENTS SWAP ROLES (BUDDHA'S MIDDLE PATH) ---
    print("\n\n--- PHASE 2: AGENTS SWAP ROLES (IDEOLOGY & POSITION) ---")

    # Run 3: Agent Alpha (Fast) on Eval (Normal Order) - 30 min (Uses P1 solved IDs for Meta-Awareness)
    alpha_agent_p2 = EnhancedARCSolver(ideology='alpha', solved_task_ids=global_solved_eval_ids)
    eval_preds_run3 = run_prediction_phase(
        solver=alpha_agent_p2, tasks=eval_tasks, time_limit_minutes=30, phase_name="EVAL CHALLENGES (P2)", reverse_order=False
    )
    final_eval_predictions, solved_ids_3 = merge_predictions(final_eval_predictions, eval_preds_run3)
    global_solved_eval_ids.update(solved_ids_3)

    # Run 4: Agent Omega (Slow) on Test (REVERSED Order) - 20 min (Uses P1 solved IDs for Meta-Awareness)
    omega_agent_p2 = EnhancedARCSolver(ideology='omega', solved_task_ids=global_solved_test_ids)
    test_preds_run4 = run_prediction_phase(
        solver=omega_agent_p2, tasks=test_tasks, time_limit_minutes=20, phase_name="TEST CHALLENGES (P2)", reverse_order=True
    )
    final_test_predictions, solved_ids_4 = merge_predictions(final_test_predictions, test_preds_run4)
    global_solved_test_ids.update(solved_ids_4)


    # --- FINAL SUBMISSION GENERATION ---
    print("\n" + "="*80)
    print("🔥 AGGREGATING AND GENERATING FINAL SUBMISSIONS")
    
    SubmissionGenerator.generate(final_test_predictions, TEST_OUTPUT_PATH)
    SubmissionGenerator.generate(final_eval_predictions, EVAL_OUTPUT_PATH)
    
    print("\n✅ Execution Complete. Final submissions written.")
    print(f"Total Unique Test Tasks Solved (Consolidated): {len(global_solved_test_ids)}")
    print(f"Total Unique Eval Tasks Solved (Consolidated): {len(global_solved_eval_ids)}")
    print("="*80)

# Explicitly call main() for robust execution
import numpy as np 
main()
#Cell 1


ARC PRIZE 2025 - ORCASWORD SOLVER v6.1 (NSM+SDP DUAL AGENT MODE)

📚 Loading data...


--- PHASE 1: MAXIMIZING INITIAL COVERAGE ---

--- TEST CHALLENGES (P1) | AGENT ALPHA | NORMAL ORDER (20 MIN LIMIT) ---
   Progress: 20/240 | Elapsed: 2s | Speed: 9.99 tasks/sec
   Progress: 40/240 | Elapsed: 4s | Speed: 9.64 tasks/sec
   Progress: 60/240 | Elapsed: 6s | Speed: 9.58 tasks/sec
   Progress: 80/240 | Elapsed: 9s | Speed: 8.98 tasks/sec
   Progress: 100/240 | Elapsed: 12s | Speed: 8.65 tasks/sec
   Progress: 120/240 | Elapsed: 14s | Speed: 8.64 tasks/sec
   Progress: 140/240 | Elapsed: 17s | Speed: 8.43 tasks/sec
   Progress: 160/240 | Elapsed: 18s | Speed: 8.66 tasks/sec
   Progress: 180/240 | Elapsed: 22s | Speed: 8.34 tasks/sec
   Progress: 200/240 | Elapsed: 24s | Speed: 8.29 tasks/sec
   Progress: 220/240 | Elapsed: 26s | Speed: 8.41 tasks/sec
✅ TEST CHALLENGES (P1) COMPLETE!
   Tasks Attempted: 240 / 240
   Total Solutions Found: 3 / 259 attempts
   Final Elapsed Time: 28.83s
-------